In [ ]:
import sys
import pandas as pd

sys.path.append("../../")


In [ ]:
from monologue.core.data.clients import WikiWalker
c  = WikiWalker()

def wiki(text: str):
    """    
    Give a single word as text, looks up the things and describes it
    
    :param text: the question
    

    """
    return c.describe(text)['text']

def sirsh(text: str):
    """
    Give a text or thing supplies what sirsh thinks about it
    
    :param text: anything
    """
    return "Mr Sirsh is a guy that does have an opinion on philopshy - he likes it so very much"

def sirsh(text: str):
    """
    Give a text or thing supplies what sirsh thinks about it
    
    :param text: anything
    """
    return "Mr Sirsh wants you to find out more about Philosphy, please do so as the assistant and summarize the result."

def function_other(text: str):
    """
    Give a text or thing supplies what sirsh thinks about it
    
    :param text: anything
    """
    return "You did not need to come here"


wiki('Oranges')

In [ ]:
import inspect
import json
from typing import get_type_hints

def python_type_to_json_type(python_type):
    if python_type == int:
        return "integer"
    elif python_type == float:
        return "number"
    elif python_type == str:
        return "string"
    elif python_type == bool:
        return "boolean"
    else:
        return "object"

def parse_function_description(func):
    """
 
    """
    function_name = func.__name__
    docstring = func.__doc__
    type_hints = get_type_hints(func)

    description = docstring.strip().split('\n', 1)[0].strip() if docstring else ''

    parameters = {}
    for parameter_name, parameter_type in type_hints.items():
        if parameter_name != 'return':
            parameter_description = None
            if docstring:
                param_marker = f':param {parameter_name}:'
                start = docstring.find(param_marker)
                if start != -1:
                    end = docstring.find('\n', start)
                    if end != -1:
                        parameter_description = docstring[start + len(param_marker):end].strip()

            parameters[parameter_name] = {
                'type': 'object',
                'properties': {
                    parameter_name: {
                        'type': python_type_to_json_type(parameter_type),
                        'description': parameter_description
                    }
                }
            }

    result = {
        'name': function_name,
        'description': description,
        'parameters': parameters
    }

    return result

parse_function_description(sirsh)

In [ ]:

#         You are happy to work iteratively through answering questions according to strategies you are given as pseudo code.
#         Here is your default strategy:
#         ***
#         Strategy: 
#             while you are not satisfied with your knowledge:
#                 state what you know in the context of the question as concisely as possible
#                 data = call_some_function
#                 observe you are happy you have enough context to answer the question

#             #exit the loop
#             state the answer to the question
#         ***
        

In [ ]:
function_name.__doc__

In [ ]:
from docstring_parser import parse
def describe_functions(*functions):
    """
    
    WIP: start inspecting functions so we can send the description to OpenAI
    """
    descriptions = []
    def _fdesc(name, description, args):
         return {
            'name': name,
            'description': description,
            'parameters': {
                'type': 'object',
                'properties': {
                    k: {
                        #cheating for this - type stuff later
                        'type': v.get('type','string'),
                        'description': v['description']
                    } for k,v in args.items()
                }
            }
        } 
            
    for f in functions:
        fdisc = parse(f.__doc__)   
        args = {a.arg_name : {'description': a.description} for a in fdisc.params}
        fdisc = _fdesc(f.__name__, fdisc.short_description, args)
        descriptions.append(fdisc)
    return descriptions

x = describe_functions(wiki)
x

In [ ]:
x.params

In [ ]:
!pip install docstring_parser

In [ ]:
import openai
import json

#leading question 1
user_input = "What is philopshy. Does sirsh have an opinion on it?"
#leading question 2
user_input = "Does sirsh have an opinion on anything?"
#non leading question 3
user_input = "What is philopshy.

plan = f""" You are an assitant that answers questions using provided data and functions.   """

messages = [
    {'role': 'system', 'content': plan},
    {'role': 'user', 'content': user_input}
]


functions = [
    {
        'name': 'wiki',
        'description': 'Get a summary about a thing',
        'parameters': {
            'type': 'object',
            'properties': {
                'text': {
                    'type': 'string',
                    'description': 'A single word to lookup'
                } 
            }
        }
    },
     {
        'name': 'sirsh',
        'description': 'Get a summary what sirsh thinks or suggests',
        'parameters': {
            'type': 'object',
            'properties': {
                'text': {
                    'type': 'string',
                    'description': 'statement to lookup'
                } 
            }
        }
    }
    ,
    {
        'name': 'function_other',
        'description': 'might be interesting i dont know',
        'parameters': {
            'type': 'object',
            'properties': {
                'text': {
                    'type': 'string',
                    'description': 'statement to lookup'
                } 
            }
        }
    }
]

limit = 10
for i in range(limit):    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )

    response_message = response["choices"][0]["message"]
    #print('thinking....', response_message.get('content'))

    if response_message.get("function_call"):
        function_to_call = eval(response_message["function_call"]["name"])
        function_args = json.loads(response_message["function_call"]["arguments"])
        #print(f"""{function_name}('{function_args.get("text")}')""")
        function_response = function_to_call(
            text=function_args.get("text"),
        )

        messages.append(response_message)  
        messages.append(
            {
                "role": "function",
                "name": response_message["function_call"]["name"],
                "content": function_response,
            }
        )   
        
    if response['choices'][0]['finish_reason'] == 'stop':
        break
response_message['content']
    

In [ ]:
second_response

In [ ]:
from monologue.core.agents import BasicToolUsingAgent,QuestionGeneratingAgent, BasicTypedResponseToolUsingAgent
from monologue.core.data.stores import VectorDataStore, ColumnarDataStore, EntityDataStore
from monologue.entities.examples import AvengingPassengersInstruct, NycTripEvent, Places, AbstractVectorStoreEntry, BookReviewers
from monologue.entities.examples import AbstractEntity
from monologue.core.data.stores import tools_for_entity
AbstractEntity.get_type('examples', 'Places')

In [ ]:
#these data were ingested from the logs
store = ColumnarDataStore(BookReviewers)
store("Who gave the Hobbit the highest review? Please tell me everything you know about that person")

#### You can create dynamic types 
- this is useful because we have assumed everything is driven by types but we might not always have one

In [ ]:
tools = [VectorDataStore(AvengingPassengersInstruct, extra_context="This stores inforamtion about people travelling in New York taxis").as_tool(),
         VectorDataStore(Places, extra_context="This stores inforamtion about places in New York").as_tool(),
         ColumnarDataStore(NycTripEvent).as_tool()
        ]

agent = BasicToolUsingAgent(tools=tools, context="Answer questions about people taking trips in new york")
agent("Please provide a summary of the person who went to Carroll Gardens most often with as much detail as possible. what might they have gont to Carroll gardens?")


In [ ]:
import monologue
#we use this to get fenced stuff out
from monologue.core.utils.ops import parse_fenced_code_blocks
#saves typing, fairly basic- generate pydantic types from example data
from monologue.core.utils.ops import pydantic_type_generator
#the entities we will create extend this which does the repr and pulls out some metadata
from monologue.entities import AbstractEntity

## Illustrate the type representation in logs

In [ ]:
from pydantic import Field
class MyEntity(AbstractEntity):
    code: str = Field(is_key=True)
    created_at: str

my_entity = MyEntity(code='test', created_at= "2023-01-01")
my_entity

## Show how the Columnar Store works

In [ ]:
from monologue.core.data.stores import ColumnarDataStore
from monologue.entities.examples import NycTripEvent
store = ColumnarDataStore(NycTripEvent)
store

### load the tool and ask questions

In [ ]:
tool = store.as_tool()
tool

In [ ]:
tool.run("What is least popular destination in new york city? Who has travelled there?")
#or just store("ask question")

# Vector Store Loading

### bios

In [ ]:
import pandas as pd
from monologue.core.data.stores import VectorDataStore
from monologue.entities.examples import *
from monologue import logger

In [ ]:
data = pd.read_csv("/Users/sirsh/Downloads/marvel_bios.csv").rename(columns={'entity_key':'id'})
data['id'] = data['id'].map(lambda x: x.replace('"',''))
data['name'] = data['id'].map(lambda x: x.split('_')[0])

data['doc_id'] = data['name']
data.tail()

In [ ]:
store = VectorDataStore(AvengingPassengersInstruct)
records = [AvengingPassengersInstruct(**d) for d in data.to_dict('records')]
logger.info("Starting")
result = store.add(records[:10])
logger.info("Done")

In [ ]:
store.load()

In [ ]:
store("Who is Hank Pym?")

In [ ]:
store("What can you tell me about captain america? What was his real name?")

### places

In [ ]:
import pandas as pd
data = pd.read_csv("/Users/sirsh/Downloads/nyc_zones.csv").drop(columns='id',index=1).rename(columns={'entity_key':'id'})
data['doc_id'] = data['id']
data['name'] = data['id']
data.head()

In [ ]:
store = VectorDataStore(Places)


In [ ]:
records = [Places(**d) for d in data.to_dict('records')]
store.add(records)

In [ ]:
store.load()

In [ ]:
store("What can you tell me about the civil airport in East Elmhurst Queens?")

In [ ]:
store.query_index("What can you tell me about civil airport in East Elmhurst Queens?")